# Peripherals and the PModIO overlay

The Zybo board can be extended with additional peripherals by connecting to the PMod interfaces available on the board. PMod stands for Peripheral Module, and can be used to connect periperals and other accessories to the board. 

PMod peripherals are accessory boards that plug directly into the PMod port and are available from  [Digilent Inc](http://www.digilentinc.com) and other suppliers. 

### Pmod peripherals



## Supported peripherals
Pynq current supports a number of PMods which can be found in the pynq documentation > [pynq.pmods package](pynq.pmods.html)

As the PMOD interface connects to FPGA pins, any accessory or peripheral supported by the FPGA IO standard (i.e. that works at the correct voltages) that can be wired or connected to a PMod port can be used.
Pynq current supports a number of Grove peripherals which can be connected to a PMod port through and interface board. 

Other peripherals can be supported by writing your own C driver  

## Pmod Connector
Each PMod connector has 12 pins (2 rows of 6 pins, where each row has 3.3V (VCC), ground (GND) and 4 data pins giving 8 data pins in total). 

### Close-up of a 12-pin Pmod



### Pmod pin numbers and assignment



Note the PMod pins are labelled 1- 7. These are the data pins. In the Digilent documentation the pins are labelled 1-12 and include the VCC and GND pins, but this should be ignored when programming in Pynq and 0-7 should be used to reference data pins.)

### Pmod-Grove Adapter

With a Pmod-to-Grove adapter a wide range of Grove peripherals can be connected to the board.  Each Grove connector has 4 pins, one each for VCC and GND leaving 2 pins on each connector for signal wires.



## Pmod Pin configuration

PMods have different pin configurations depending on the number of data pins required. PMod peripherals with only a single row of pins can be plugged into the top row or the bottom row of the PMod connector.

Some PMods use both rows (2x4, 2x6 pins), and in general should be aligned to the uppoer-left of the connector (to align with VCC and GND).



## IO Processors
For overlays to be useful, they must provide sufficient functionality, while also providing flexibility to suit a wide range of applications. Flexibility in the overlay is provided through IO Processors (IOPs). An IO Processor is implemented in the Programmable Logic (PL or FPGA fabric) and consists of a MicroBlaze processor, Memory, Timer, IIC, SPI, GPIO, Debug logic and a configurable switch.



The IOP gives flexibility to connect to and control a range of different external accessories without requiring a re-design of the FPGA hardware. The IOP can be used to control a PMod port, or other interface, or to off-load applications from the main ARM A9 processors in the Zynq PS.

Each IOP has a dedicated memory block for the MicroBlaze instruction and data memory. This memory block is dual-port, with one port connected to the MicroBlaze, and the other connected to the Zynq A9 processors. This allows the ARM A9 processors to access the MicroBlaze memory and dynamically write a new program to the MicroBlaze instruction area. The data area can be used for communication and data transfer between the A9 processors and the IOP(s).

Currently, one IOP can support one PMod port. Inside the IOP are dedicated interface blocks; Timer, IIC, SPI and GPIO (General Purpose Input/Output). IIC and SPI are specialized interfaces used by many of the available PMod, Grove and other peripherals, and GPIO can be used to connect to custom interfaces or used as simple inputs and outputs. When a PMod or other peripheral is plugged in to a port, the configurable switch allows the PMod signals to be routed internally to the required interface block.

## ZYBO Pmodio Overlay

The ZYBO has 6 PMod connectors. Ports 1-4 as indicated below are connected the Zynq PL. XADC is connected to the Xilinx ADC on the Zynq, and JF is connected directly to the Zynq PS. XADC and JF are not included in this overlay.




The PModIO overlay for the ZYBO contains 4 IOPs each connected to one of the PL PMod ports. This allows control of the PMod from Python, via the IOP.




A PMod module is provided in Python, the module must be imported. For PMod modules, the port that the PMod is plugged into must also be specified.

### Example: Using the OLED and the ambient light sensor (ALS)

Plug the PModOLED into Port 1, and PModALS (Ambient Light Sensor) into the top row of Port 2. (The PModALS can currently only be used in the top row of a port.)

#### OLED displaying light reading from ambient light sensor



Run the following code to import the OLED module and instantiate it:

In [1]:
from pynq.pmods import OLED
oled = OLED(1)

ImportError: No module named 'pynq'

Try writing a message to the OLED.

In [ ]:
oled.write("Hello World")

In [ ]:
import time
time.sleep(2) # Add delay to make OLED output visible for a few seconds
              # if running whole workbook

In [ ]:
oled.clear_screen()

Import the ALS library, create an instance, and read the value from the sensor

In [ ]:
from pynq.pmods import ALS
als = ALS(2)
als.read()

Write the value from the ALS to the OLED. The ALS sensor returns an 8-bit value. 0 = Darkest, 255 = Brightest

In [ ]:
oled.write("Light value : " + str(als.read()))

In [ ]:
from pynq.pmods import ALS
import time
als = ALS(2)
als.read()

als.set_log_interval_ms(100)

als.start_log() # start logging values at intervals of delay
time.sleep(.2)
als.stop_log()
als.get_log()

### Example: Using the ADC, DAC and OLED
This example connects the DAC to the ADC. You will write a value to the DAC which the ADC will measure, and also write the value to the OLED for display.

* Set the jumper (JP1) on the PModAD2 to Ref

Insert the following PMods:
* OLED to Port 1
* (ALS can be left plugged in to port 2 if used earlier, but is not used for this demo)
* ADC to Port 3 (Align to the left of the port)
* DAC to Port 4 (Top Row)


* Connect a wire from PModA4 Pin A to PModAD2 V1



* Power on the board, and open the Pynq portal [pynq:9090](http://pynq:9090)
* Run Python script below.

Note the overlay will be reloaded to reset the IOPs if the code at the top of this notebook was run. You can change the value written to the DAC (must be between 0 and 2.0) and rerun the cell. The return value from the ADC and displayed on the LED is the measured value. There may be some small error between this and the value written to the DAC. 

In [ ]:
from pynq.pmods import OLED, ADC, DAC
from pynq import Overlay
ol = Overlay("pmod.bit")
ol.download()

oled = OLED(1)
adc = ADC(3)
dac = DAC(4)

dac.write(0.75) # Set the DAC output voltage
value = adc.read() # Measure output of DAC with ADC
oled.write(value) # Display measured value on OLED

For documentation on other supported peripherals, see 

* [PMods pynq.pmods package](pynq.pmods.html)



## Going further
Scripts can be created and executed from the notebook. 

In this example, the code below will be copied to a script and saved on the board. Examine the code in the cell below, and copy it. 

Go back to the Pynq Portal [pynq:9090](http://pynq:9090)
* Select New > Text File (which opens in a new window)
* Paste the code into the file
* Rename the file dac_loop.py, and save it

In [ ]:
from pynq.pmods import OLED, ADC, DAC
from pynq import Overlay
import time
# dac_loop.py
def dac_loop(DAC_Port = 4, ADC_Port = 3, OLED_Port = 1,Switch_nmbr = 3, delay_secs = 1):
    '''
    DAC -> ADC -> OLED -- LOOP OVER 10 Values
    '''
    ol = Overlay("pmod.bit")
    ol.download()
    # Import PMOD classes from library PythonXilinx (pynq.pmods)
    from pynq.pmods import OLED, ADC, DAC

    from pynq.board import  LED, Switch

    # Match PMOD instances to PMOD Connectors

    # Instantiate OLED
    oled = OLED(OLED_Port)

    # Instantiate Digital to Analog Converter
    dac = DAC(DAC_Port)

    # Instantiate Analog to Digital Converter
    adc = ADC(ADC_Port)

    LED(Switch_nmbr).write(0)
    
    if(Switch(Switch_nmbr).read() == 0):
        print("Switch 3 is off. To run the loop, set Switch 3 to on before running")
    else:
        print("Turn Switch 3 off to stop the program")
    
    while Switch(Switch_nmbr).read():
        LED(Switch_nmbr).write(1)
        for i in range (1, 11):
            dac.write(i/10)
            measured = adc.read()
            time.sleep(.1)
            print('Measurement {} equals: {}'.format(i, measured))
            oled.write(measured)
    LED(Switch_nmbr).write(0)
    

By default, this uses the previous configuration (DAC connected to Port 1 (Top Row), ADC connected to Port 2, OLED connected to Port 4 and a wire connecting the DAC to the ADC). The function reads  _Switch_ 4 on the board. If the switch is on, it will run a loop that writes to the DAC, reads the value on the ADC and writes to the OLED. This is the same as the previous example, but this time executing from a script, and inside a loop controlled by a switch.

To run this code, the cell above can be executed, which loads dac_loop(), or the cell below can directly execute the script. 

Try executing the cell above, set Switch 4 on the board, and then run the cell below:

In [ ]:
dac_loop()

Restart the kernel (Kernel > Restart) to clear the dac_loop function, execute the cells below to run the script directly, and execute the dac_loop() function 

In [ ]:
%run dac_loop.py
dac_loop()